# Bedrock AgentCore Gateway を使用して OpenAPI を MCP ツールに変換する

## 概要
Bedrock AgentCore Gateway は、お客様が既存の API をインフラやホスティングを管理することなく、フルマネージドの MCP サーバーに変換する方法を提供します。お客様は JSON または YAML 形式の OpenAPI スペックを持ち込むことができます。ここでは、OAuth2 で保護されたエンタープライズサポート API を使用するカスタマーサービスエージェントを実演します。

Gateway のワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます：
* **Gateway 用のツールを作成** - REST API 用の OpenAPI 仕様などのスキーマを使用してツールを定義します。OpenAPI 仕様は Amazon Bedrock AgentCore によって解析され、Gateway が作成されます。
* **Gateway エンドポイントを作成** - インバウンド認証を備えた MCP エントリポイントとして機能するゲートウェイを作成します。
* **Gateway にターゲットを追加** - ゲートウェイがリクエストを特定のツールにルーティングする方法を定義する OpenAPI ターゲットを設定します。OpenAPI ファイルに含まれるすべての API が MCP 互換ツールとなり、Gateway エンドポイント URL を通じて利用可能になります。各 OpenAPI Gateway ターゲットに OAuth を使用したアウトバウンド認可を設定します。
* **エージェントコードを更新** - 統合された MCP インターフェースを通じてすべての設定済みツールにアクセスするため、エージェントを Gateway エンドポイントに接続します。

![仕組み](images/openapis-oauth-gateway.png)

### チュートリアル詳細

| 情報                  | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ    | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agent                                             |
| Gateway ターゲットタイプ | OpenAPI                                                   |
| エージェント           | カスタマーサポートエージェント                              |
| インバウンド認証 IdP    | Okta                                                      |
| アウトバウンド認証      | OAuth                                                     |
| LLM モデル             | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| チュートリアル構成      | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル業界      | クロスバーティカル                                         |
| 難易度                 | 簡単                                                       |
| 使用 SDK               | boto3                                                     |

チュートリアルの最初のパートでは、AgentCore Gateway ターゲットを作成します。

### チュートリアルアーキテクチャ
このチュートリアルでは、OpenAPI yaml/json ファイルで定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモンストレーション目的で、サポートチケットに関するクエリに回答するカスタマーサポートエージェントを構築します。エージェントは Zendesk サポート API の OpenAPI を使用します。ソリューションは Amazon Bedrock モデルを使用した Langchain エージェントを使用します。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python 3.10+）
* uv
* AWS 認証情報
* Okta
    - client_id
    - client_secret
    - Okta ドメイン（例：dev-123456.okta.com）
    - OAuth2 認可サーバー ID（通常は default）
* Okta と統合された Zendesk

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using SageMaker notebooks
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore Gateway ロール ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認可用に Okta を設定

Okta で OAuth オーソライザーを作成するには、以下の手順に従ってください。

* Okta サブスクリプションをお持ちでない場合は、[こちら](https://www.okta.com/free-trial/)で無料トライアルにサインアップしてください。
* Okta 管理コンソールにサインインします。
* [こちら](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/)の手順に従って、**Client Credentials** グラントタイプのアプリケーションを作成します。
* アプリケーション作成後、アプリケーションページに移動し、作成したアプリケーションを選択します。**Client ID** と **Client Secret** をテキストエディタに保存します。
* General Settings で **Require Demonstrating Proof of Possession (DPoP)** を無効にします。
* 左のナビゲーションバーを開き、Security -> API を選択します。作成されたデフォルト認可サーバーを選択します。
* **Audience** の値をテキストエディタに保存します。
* **Issuer** の値（`https://trial-xxxxx.okta.com/oauth2/default` のような形式）をテキストエディタに保存します。
* カスタムスコープを定義します。Scopes タブに移動し、「Add Scope」をクリックします。InvokeGateway というスコープを追加します。
* **Access Policies** を選択します。新しいアクセスポリシーを作成します。名前を付け、All Clients に割り当てます。ポリシー作成後、**Add Rule** を選択し、すべての値をデフォルトのままにして **Create Rule** を選択します。

# インバウンド認可用に Okta オーソライザーを使用して Gateway を作成

In [ ]:
import boto3
from pprint import pprint
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

OKTA_DISCOVERY_URL="<Your Okta Issuer value>/.well-known/openid-configuration"
OKTA_AUDIENCE="<The audience value you saved earlier>" 

auth_config = {
        "customJWTAuthorizer": {
            "allowedAudience": [OKTA_AUDIENCE],
            "discoveryUrl": OKTA_DISCOVERY_URL
        }
}
create_response = gateway_client.create_gateway(name='OpenAPIOktaGwy2',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway created from sdk with Okta Authorizer'
)
pprint(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]

# Bedrock AgentCore Gateway を使用して Zendesk サポート API を MCP ツールに変換

### アウトバウンド認証用の認証情報プロバイダーを作成

In [ ]:
from botocore.config import Config
ZENDESK_DOMAIN="<Zendek domain url>"
ZENDESK_AUTH_ENDPOINT="https://<Zendeskl-domain>/oauth/authorizations/new"
ZENDESK_TOKEN_ENDPOINT="https://<Zendesk-domain>/oauth/tokens"
ZENDESK_CLIENT_ID="" # Your Zendesk OAuth client -  client id 
ZENDESK_SECRET=""  # Your Zendesk OAuth client -  client id 

sdk_config = Config(
    region_name=os.environ['AWS_DEFAULT_REGION'],
    retries={"max_attempts": 2, "mode": "standard"},
)

acps = boto3.client(
    service_name="bedrock-agentcore-control",
    config=sdk_config,
)

provider_config= {
    "customOauth2ProviderConfig": {
         "oauthDiscovery": {
             "authorizationServerMetadata": {
                 "issuer": ZENDESK_DOMAIN,
                 "authorizationEndpoint": ZENDESK_AUTH_ENDPOINT,
                 "tokenEndpoint": ZENDESK_TOKEN_ENDPOINT,
                 "responseTypes": ["token"]
             }
         },
         "clientId": ZENDESK_CLIENT_ID,
         "clientSecret": ZENDESK_SECRET
     }
 }

response = acps.create_oauth2_credential_provider(
    name="ZendeskOAuthTokenCfg", 
    credentialProviderVendor="CustomOauth2", 
    oauth2ProviderConfigInput=provider_config
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress クレデンシャルプロバイダー ARN: {credentialProviderARN}")

### OpenAPI ターゲットを作成

#### Zendesk サポート OpenAPI yaml ファイルを S3 にアップロード

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.
file_path = 'openapi-specs/Zendesk-support-apis.yaml'
object_key = 'Zendesk-support-apis.yaml'
# Upload the file using put_object and read response
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # Construct the ARN of the uploaded object with account ID and region
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'アップロードされたオブジェクトの S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'ファイルアップロードエラー: {e}')

#### Gateway ターゲットを作成

OpenAPI ファイルのサーバー URL が独自のエンドポイント URL を指していることを確認してください。Gateway は OpenAPI ファイルからサーバー URL を読み取り、エンドポイントを呼び出します。S3 にアップロードする前に、必ずこの変更を行ってください。

In [ ]:
# S3 Uri for OpenAPI spec file
openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

credential_config = [
    {
        "credentialProviderType" : "OAUTH",
        "credentialProvider": {
            "oauthCredentialProvider": {
                "providerArn": credentialProviderARN, 
                "scopes": ["tickets:read", "read", "tickets:write", "write"] 
            }
        }
    }
  ]

target_name="DemoOpenAPIGW"
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=target_name,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=openapi_s3_target_config,
    credentialProviderConfigurations=credential_config)

# Printing the request ID and timestamp for you to report the defects. Please include them while reporting issues/defects  
response_metadata = response['ResponseMetadata']

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装した Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合します。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、任意の MCP 準拠クライアントまたは SDK が利用可能なツールを発見し、安全で標準化された方法で対話できます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は [MCP 認可仕様](https://modelcontextprotocol.org/specification/draft/basic/authorization) に厳密に準拠しており、堅牢なセキュリティとアクセス制御を保証します。これは、Strands エージェントによるすべてのツール呼び出しが認可ステップを経ることを意味し、強力な機能を有効にしながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。Gateway はすべての必要なセキュリティ検証、プロトコル変換、サービス対話を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装する任意のクライアントまたは SDK が Gateway を通じて AWS サービスと対話できるため、AI エージェント統合のための汎用性が高く将来を見据えたソリューションとなります。

# インバウンド認可用に Okta からアクセストークンをリクエスト

In [ ]:
print("Okta オーソライザーからアクセストークンをリクエスト中")
import requests
from requests.auth import HTTPBasicAuth

# Replace with your actual values
OKTA_DOMAIN = "Your Okta domain URL"
AUTH_SERVER_ID = "Okta app id"
CLIENT_ID = "<Okta client credentials client id>"
CLIENT_SECRET = "<Okta client credentials secret>"

TOKEN_URL = f"{OKTA_DOMAIN}/oauth2/{AUTH_SERVER_ID}/v1/token"

response = requests.post(
    TOKEN_URL,
    auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type": "client_credentials", "scope": "InvokeGateway"}
)

if response.status_code == 200:
    token = response.json()["access_token"]
    print("アクセストークン:", token)
else:
    print("トークン取得に失敗しました:", response.status_code, response.text)

# Bedrock AgentCore Gateway を使用して Zendesk サポート API でカスタマーサポートエージェントに質問

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    #print(f"エージェントに読み込まれたツール: {agent.tool_names}")
    #print(f"エージェントのツール設定: {agent.tool_config}")
    
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    #agent("利用可能なすべてのツールをリストしてもらえますか")
    agent("サポートチケットの数をカウントしてください")
    # Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    result = client.call_tool_sync(
    tool_use_id="count-tickets-1", # You can replace this with unique identifier. 
    name="DemoOpenAPIGW___CountTickets", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    )
    #Print the MCP Tool response
    print(f"ツール呼び出し結果: {result['content'][0]['text']}")

# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されます。クリーンアップの一環として手動で削除する必要がある場合があります。これは実行する例によって異なります。

## Gateway を削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)